In [9]:
!pip install transformers accelerate bitsandbytes

In [10]:
!pip install scipy

In [11]:
#!pip install sentencepiece
!pip show sentencepiece
!pip install --upgrade sentencepiece



Name: sentencepiece
Version: 0.2.0
Summary: SentencePiece python wrapper
Home-page: https://github.com/google/sentencepiece
Author: Taku Kudo
Author-email: taku@google.com
License: Apache
Location: /home/codemaster29/miniconda3/lib/python3.12/site-packages
Requires: 
Required-by: 


In [12]:
!pip install einops torchvision

In [18]:
!pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 13.9 MB/s eta 0:00:00m eta 0:00:010:01:01


In [13]:
#!sudo pacman -S gcc -y


In [19]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 4070 SUPER


In [21]:
import torch
import requests
from PIL import Image
from transformers import AutoModelForCausalLM, LlamaTokenizer

# Load the tokenizer for the 'vicuna-7b-v1.5' model
tokenizer = LlamaTokenizer.from_pretrained('lmsys/vicuna-7b-v1.5')

# Load the pre-trained model 'THUDM/cogvlm-chat-hf' with torch bfloat16 precision and optimized for lower memory usage
model = AutoModelForCausalLM.from_pretrained(
    'THUDM/cogvlm-chat-hf',
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="balanced",
    device_map="auto"
).to('cuda').eval()  # Move model to GPU and set to evaluation mode


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacity of 11.62 GiB of which 168.81 MiB is free. Including non-PyTorch memory, this process has 10.72 GiB memory in use. Of the allocated memory 10.55 GiB is allocated by PyTorch, and 1.79 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [6]:
import requests
from PIL import Image
from io import BytesIO

def generate(query, img_url):
    # Fetch the image from the URL
    response = requests.get(img_url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content)).convert('RGB')
        
        # Continue with the model processing...
        inputs = model.build_conversation_input_ids(
            tokenizer, query=query, history=[], images=[image], template_version='vqa'
        )
        inputs = {
            'input_ids': inputs['input_ids'].unsqueeze(0).to('cuda'),
            'token_type_ids': inputs['token_type_ids'].unsqueeze(0).to('cuda'),
            'attention_mask': inputs['attention_mask'].unsqueeze(0).to('cuda'),
            'images': [[inputs['images'][0].to('cuda').to(torch.bfloat16)]],
        }
    else:
        print("Failed to fetch image from URL.")


In [8]:
query = 'Describe this image in detail'
image = 'https://m.media-amazon.com/images/I/41-NCxNuBxL.jpg'
generate(query, image)

NameError: name 'model' is not defined